# Langchain

## Prompt Templates

In [1]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})

StringPromptValue(text='Tell me a joke about cats')

## Chat Prompt Templates

In [2]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

## Type of messages

In [3]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

## Langchain OpenAI

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

## Chaining

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

# Using Images

In [6]:
import base64

with open("data/cat.jpg", "rb") as f:
    image_data = base64.b64encode(f.read()).decode()

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = llm.invoke([message])
print(response.content)

The image shows a gray cat with bright orange eyes and an open mouth, giving it an expression of surprise or curiosity. The background is a solid orange color, which contrasts with the cat's fur.


## Extracting structured information

In [7]:
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="first name of a Person")
    lastname: str = Field(description="last name of a Person")
    email: str = Field(description="Email of a Person")
    age: int = Field(description="Age of a Person")


response = llm.with_structured_output(Person).invoke([
    ("system", "you are an specialist extracting person information from a text"),
    ("user", "mi nombre es james espichan, mi corre es james@idat.com y tengo 50 años")
])

In [8]:
response

Person(name='James', lastname='Espichan', email='james@idat.com', age=50)

## Let's use a bill

In [9]:
class Item(BaseModel):
    description: str = Field(description="description for each item")
    price: float = Field(description="price of each item")

class Bill(BaseModel):
    items: list[Item] = Field(description="List of item that contains descriptions for each item and prices")

In [11]:

with open("data/factura1.png", "rb") as f:
    image_data = base64.b64encode(f.read()).decode()

In [12]:
response = llm.with_structured_output(Bill).invoke([
    ("system", "you are an specialist extracting items information from a billing"),
    HumanMessage(content=[
        {"type": "text", "text": "billing"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
    )
])

In [14]:
response.items

[Item(description='Talla pequeña traje de luces en rojo', price=100.0),
 Item(description='Mui grande churrolito', price=25.0),
 Item(description='Equipaje de Fútbol', price=5.0)]